# Cassava Leaf Disease Classification

In [ ]:
from tensorflow.keras import models, layers
from tensorflow import keras
from tensorflow.keras.applications import ResNet50, DenseNet121, EfficientNetB0
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
# tf.enable_eager_execution()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_hub as hub

In [ ]:
TRAIN_DIR = '../input/cassava-leaf-disease-classification/train_images/'
labels = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
labels.label = labels.label.astype('str')

labels.head()

In [ ]:
BATCH_SIZE = 16
IMAGE_SIZE = [512, 512]
AUTOTUNE = tf.data.experimental.AUTOTUNE

BATCH_SIZE = 8
STEPS_PER_EPOCH = len(labels)*0.8 / BATCH_SIZE
VALIDATION_STEPS = len(labels)*0.2 / BATCH_SIZE
EPOCHS = 50
TARGET_SIZE = 512

In [ ]:
generator_train = keras.preprocessing.image.ImageDataGenerator(rotation_range=90,
                                                               shear_range=0.2, 
                                                               zoom_range=0.2, 
                                                               horizontal_flip=True,
                                                               vertical_flip=True,
                                                               validation_split=0.2)

In [ ]:
train_gen = generator_train.flow_from_dataframe(labels,
                                          directory = TRAIN_DIR,
                                          subset='training',
                                          x_col = "image_id",
                                          y_col = "label",
                                          batch_size = BATCH_SIZE,
                                          class_mode = "sparse",
                                          shuffle=True)
val_gen   = generator_train.flow_from_dataframe(labels,
                                          directory = TRAIN_DIR,
                                          batch_size = BATCH_SIZE,
                                          x_col = "image_id",
                                          y_col = "label",
                                          class_mode = "sparse",
                                          subset='validation')

In [ ]:
# from internet
aug_images = [train_gen[0][0][0]/255 for i in range(10)]
fig, axes = plt.subplots(2, 5, figsize = (20, 10))
axes = axes.flatten()
for img, ax in zip(aug_images, axes):
    ax.imshow(img)
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
num_classes = 5
img_height = img_width = 512

In [ ]:
feature_extractor_model1 = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
feature_extractor_model2 = 'https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1'

In [ ]:
# first not trainable
feature_extractor_layer1 = hub.KerasLayer(
    feature_extractor_model1, input_shape=(img_width, img_height, 3), trainable=False)

feature_extractor_layer2 = hub.KerasLayer(
    feature_extractor_model1, input_shape=(img_width, img_height, 3), trainable=True)

feature_extractor_layer3 = hub.KerasLayer(
    feature_extractor_model2, input_shape=(img_width, img_height, 3), trainable=True)

In [ ]:
resnet_not = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  feature_extractor_layer1,
  tf.keras.layers.Dense(100, activation = "relu"),  
  tf.keras.layers.Dropout(0.2),  
  tf.keras.layers.Dense(100, activation = "relu"),  
  tf.keras.layers.Dropout(0.5),  
  tf.keras.layers.Dense(5, activation = "softmax")
])

resnet_not.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

resnet = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  feature_extractor_layer2,
  tf.keras.layers.Dense(100, activation = "relu"),  
  tf.keras.layers.Dropout(0.2),  
  tf.keras.layers.Dense(5, activation = "softmax")
])

resnet.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

efficient = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  feature_extractor_layer3,
  tf.keras.layers.Dense(100, activation = "relu"),  
  tf.keras.layers.Dropout(0.2),  
  tf.keras.layers.Dense(5, activation = "softmax")
])

efficient.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

In [ ]:
# from internet
model1_save = tf.keras.callbacks.ModelCheckpoint('./resnet_not_trained_v2_50.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)

model2_save = tf.keras.callbacks.ModelCheckpoint('./resnet_trained_v2_50.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)


model3_save = tf.keras.callbacks.ModelCheckpoint('./efficientnet.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)




early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)

# ResNet2 non trainable

In [ ]:
try: 
    with tf.device('/gpu:0'):
        history_resnet_not = resnet_not.fit(train_gen,
                                            validation_data = val_gen,
                                            steps_per_epoch = 1800,
                                            validation_steps = VALIDATION_STEPS,
                                            epochs = EPOCHS,
                                            callbacks=[model1_save, early_stop])
except RuntimeError as e:
    print(e)

In [ ]:
plt.plot(history_resnet_not.history['acc'])
plt.plot(history_resnet_not.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# ResNet2 trainable

In [ ]:
try: 
    with tf.device('/gpu:0'):
        history_resnet = resnet.fit(train_gen,
                                            validation_data = val_gen,
                                            steps_per_epoch = 1800,
                                            validation_steps = VALIDATION_STEPS,
                                            epochs = EPOCHS,
                                            callbacks=[model2_save, early_stop])
except RuntimeError as e:
    print(e)

In [ ]:
plt.plot(history_resnet.history['acc'])
plt.plot(history_resnet.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# EfficientNet trainable

In [ ]:
#try: 
#    with tf.device('/gpu:0'):
#        history_efficient = efficient.fit(train_gen,
#                                        validation_data = val_gen,
#                                        steps_per_epoch = 1800,
#                                        validation_steps = VALIDATION_STEPS,
#                                        epochs = EPOCHS,
#                                        callbacks=[model3_save, early_stop])
#except RuntimeError as e:
#    print(e)

In [ ]:
#plt.plot(history_efficient.history['acc'])
#plt.plot(history_efficient.history['val_acc'])
#plt.title('model accuracy')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'val'], loc='upper left')
#plt.show()